<a href="https://colab.research.google.com/github/simonyelisey/Raifhack-DS-2021/blob/main/raifhack_main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install scikit-learn==0.24

     |████████████████████████████████| 22.3 MB 1.7 MB/s 
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 0.22.2.post1
    Uninstalling scikit-learn-0.22.2.post1:
      Successfully uninstalled scikit-learn-0.22.2.post1


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os,gc
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_percentage_error
from sklearn.model_selection import train_test_split
from xgboost import XGBRegressor

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
os.chdir('/content/drive/MyDrive')

In [5]:
os.listdir('raifhack_ds')

['raifhack_ds']

In [ ]:
# данные
df_train = pd.read_csv('raifhack_ds/raifhack_ds/data/train.csv', index_col='id')
df_test = pd.read_csv('raifhack_ds/raifhack_ds/data/test.csv', index_col='id')

In [9]:
# оставим только данные с ручной оценкой стоимости
df_train = df_train[df_train['price_type'] == 1]

In [13]:
df_train.head()

,city,floor,lat,lng,osm_amenity_points_in_0.001,osm_amenity_points_in_0.005,osm_amenity_points_in_0.0075,osm_amenity_points_in_0.01,osm_building_points_in_0.001,osm_building_points_in_0.005,osm_building_points_in_0.0075,osm_building_points_in_0.01,osm_catering_points_in_0.001,osm_catering_points_in_0.005,osm_catering_points_in_0.0075,osm_catering_points_in_0.01,osm_city_closest_dist,osm_city_nearest_name,osm_city_nearest_population,osm_crossing_closest_dist,osm_crossing_points_in_0.001,osm_crossing_points_in_0.005,osm_crossing_points_in_0.0075,osm_crossing_points_in_0.01,osm_culture_points_in_0.001,osm_culture_points_in_0.005,osm_culture_points_in_0.0075,osm_culture_points_in_0.01,osm_finance_points_in_0.001,osm_finance_points_in_0.005,osm_finance_points_in_0.0075,osm_finance_points_in_0.01,osm_healthcare_points_in_0.005,osm_healthcare_points_in_0.0075,osm_healthcare_points_in_0.01,osm_historic_points_in_0.005,osm_historic_points_in_0.0075,osm_historic_points_in_0.01,osm_hotels_points_in_0.005,osm_hotels_points_in_0.0075,osm_hotels_points_in_0.01,osm_leisure_points_in_0.005,osm_leisure_points_in_0.0075,osm_leisure_points_in_0.01,osm_offices_points_in_0.001,osm_offices_points_in_0.005,osm_offices_points_in_0.0075,osm_offices_points_in_0.01,osm_shops_points_in_0.001,osm_shops_points_in_0.005,osm_shops_points_in_0.0075,osm_shops_points_in_0.01,osm_subway_closest_dist,osm_train_stop_closest_dist,osm_train_stop_points_in_0.005,osm_train_stop_points_in_0.0075,osm_train_stop_points_in_0.01,osm_transport_stop_closest_dist,osm_transport_stop_points_in_0.005,osm_transport_stop_points_in_0.0075,osm_transport_stop_points_in_0.01,per_square_meter_price,reform_count_of_houses_1000,reform_count_of_houses_500,reform_house_population_1000,reform_house_population_500,reform_mean_floor_count_1000,reform_mean_floor_count_500,reform_mean_year_building_1000,reform_mean_year_building_500,region,total_square,street,date,realty_type,price_type
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
COL_62,Красноярск,NaN,56.063615,92.958428,0,7,14,26,0,0,0,0,0,0,0,2,7.795659,Красноярск,1095286.0,0.041323,2,6,25,35,0,0,0,2,0,1,2,3,0,0,0,0,0,1,0,0,0,1,1,1,0,0,0,0,0,5,10,15,1.087101,2.281425,0,0,0,0.052809,7,15,21,103120.697310,95,31,2670.0,940.0,7.136842,7.612903,1976.684211,1980.483871,Красноярский край,106.351104,S5845,2020-01-05,110,1
COL_71,Саратов,NaN,51.534581,46.020549,13,198,345,462,0,0,0,0,4,44,66,89,0.987160,Саратов,842097.0,0.144936,0,4,34,59,0,1,6,9,0,13,22,27,7,17,27,4,7,9,2,5,7,11,16,20,1,11,21,28,8,125,218,286,357.425239,1.924421,0,0,0,0.045451,22,34,45,24874.371004,502,104,4886.0,943.0,4.144608,4.164557,1940.591549,1939.348837,Саратовская область,175.039602,S27968,2020-01-05,10,1
COL_140,Красноярск,NaN,56.026884,92.818323,3,15,23,33,0,0,0,0,0,1,3,4,3.730568,Красноярск,1095286.0,0.141526,0,16,35,56,0,0,0,0,2,3,3,3,0,1,1,3,3,3,0,0,1,2,2,2,0,2,2,3,1,5,10,18,1.621353,0.579096,0,0,2,0.223787,6,11,18,80058.148547,175,66,4282.0,1445.0,6.851429,5.742424,1976.714286,1972.878788,Красноярский край,89.534921,S23597,2020-01-05,10,1
COL_202,Иркутск,NaN,52.275528,104.251444,0,10,26,40,0,1,1,1,0,2,6,8,2.344310,Иркутск,623562.0,0.281491,0,6,6,6,0,0,0,0,0,0,0,1,0,4,4,0,0,0,0,1,2,1,2,2,0,1,1,1,0,5,11,21,61.562546,0.813594,0,0,2,0.266067,8,11,19,24649.765045,230,56,1887.0,612.0,4.597222,5.469388,1968.098131,1975.530612,Иркутская область,419.557752,S10587,2020-01-05,10,1
COL_207,Белгород,NaN,50.576545,36.584197,4,48,73,92,0,0,6,6,1,7,11,15,2.170184,Белгород,384425.0,0.065996,1,19,28,46,1,5,10,11,0,2,2,4,3,3,4,2,2,2,0,2,2,10,11,15,0,0,0,0,1,19,29,36,555.411948,2.505330,0,0,0,0.092583,7,17,22,39135.228322,131,35,3837.0,1073.0,9.157480,9.088235,1983.125984,1980.470588,Белгородская область,1098.703185,S11220,2020-01-05,10,1


## Обработка floor

В признаке **floor** очень много мусора, обработаем его, приведя к числовому формату, а также создадим новый признак, показывающий количество занимаемых помещением этажей

In [21]:
df_train.floor.unique()

array([nan, '2.0', '1', '-1.0', '1.0', '3.0', '18.0', 'подвал, 1', '2',
       '4.0', 'подвал', 'цоколь, 1', '1,2,антресоль', '5.0', '6.0',
       'цоколь', '9.0', '4', '5', 'тех.этаж (6)', '3', 'Подвал', 'Цоколь',
       '10', 'фактически на уровне 1 этажа', '6', '1,2,3', '1, подвал',
       '1,2,3,4', '1,2', '1,2,3,4,5', '5, мансарда', '-2.0', '10.0',
       '1-й, подвал', '12', '15', '13', '1, подвал, антресоль', '16.0',
       'мезонин', 'подвал, 1-3', '8', '7', '15.0', '1 (Цокольный этаж)',
       '3, Мансарда (4 эт)', 'подвал,1', '1, антресоль', '1-3',
       'мансарда (4эт)', '1, 2.', '9', '7.0', 'подвал , 1 ', '1, 2',
       'подвал, 1,2,3', '0.0', '1 + подвал (без отделки)', 'мансарда',
       '2,3', '4, 5', '1-й, 2-й', '18', '1 этаж, подвал', '17.0',
       '1, цоколь', 'подвал, 1-7, техэтаж', '3 (антресоль)', '1, 2, 3',
       'Цоколь, 1,2(мансарда)', 'подвал, 3. 4 этаж', 'подвал, 1-4 этаж',
       'подва, 1.2 этаж', '2, 3', '-1', '1.2', '11', '36', '7,8',
       '1 этаж', '

### Train Признак с количеством этажей

In [22]:
df_train['amnt_floor'] = None

In [23]:
df_train.loc[df_train['floor'] == '2.0', 'amnt_floor'] = 1
df_train.loc[df_train['floor'] == '1', 'amnt_floor'] = 1
df_train.loc[df_train['floor'] == '-1.0', 'amnt_floor'] = 1
df_train.loc[df_train['floor'] == '1.0', 'amnt_floor'] = 1
df_train.loc[df_train['floor'] == '3.0', 'amnt_floor'] = 1
df_train.loc[df_train['floor'] == '18.0', 'amnt_floor'] = 1
df_train.loc[df_train['floor'] == '2', 'amnt_floor'] = 1
df_train.loc[df_train['floor'] == '4.0', 'amnt_floor'] = 1
df_train.loc[df_train['floor'] == '5.0', 'amnt_floor'] = 1
df_train.loc[df_train['floor'] == '6.0', 'amnt_floor'] = 1
df_train.loc[df_train['floor'] == '9.0', 'amnt_floor'] = 1
df_train.loc[df_train['floor'] == '4', 'amnt_floor'] = 1
df_train.loc[df_train['floor'] == '5', 'amnt_floor'] = 1
df_train.loc[df_train['floor'] == '3', 'amnt_floor'] = 1
df_train.loc[df_train['floor'] == '10', 'amnt_floor'] = 1
df_train.loc[df_train['floor'] == '6', 'amnt_floor'] = 1
df_train.loc[df_train['floor'] == '-2.0', 'amnt_floor'] = 1
df_train.loc[df_train['floor'] == '10.0', 'amnt_floor'] = 1
df_train.loc[df_train['floor'] == '12', 'amnt_floor'] = 1
df_train.loc[df_train['floor'] == '15', 'amnt_floor'] = 1
df_train.loc[df_train['floor'] == '13', 'amnt_floor'] = 1
df_train.loc[df_train['floor'] == '16.0', 'amnt_floor'] = 1
df_train.loc[df_train['floor'] == '8', 'amnt_floor'] = 1
df_train.loc[df_train['floor'] == '7', 'amnt_floor'] = 1
df_train.loc[df_train['floor'] == '15.0', 'amnt_floor'] = 1
df_train.loc[df_train['floor'] == '9', 'amnt_floor'] = 1
df_train.loc[df_train['floor'] == '7.0', 'amnt_floor'] = 1
df_train.loc[df_train['floor'] == '0.0', 'amnt_floor'] = 1
df_train.loc[df_train['floor'] == '18', 'amnt_floor'] = 1
df_train.loc[df_train['floor'] == '17.0', 'amnt_floor'] = 1
df_train.loc[df_train['floor'] == '-1', 'amnt_floor'] = 1
df_train.loc[df_train['floor'] == '11', 'amnt_floor'] = 1
df_train.loc[df_train['floor'] == '36', 'amnt_floor'] = 1
df_train.loc[df_train['floor'] == '29', 'amnt_floor'] = 1

In [24]:
df_train.loc[df_train['floor'] == '1, 2, 3', 'amnt_floor'] = 3
df_train.loc[df_train['floor'] == '1,2', 'amnt_floor'] = 2
df_train.loc[df_train['floor'] == '1, подвал', 'amnt_floor'] = 2
df_train.loc[df_train['floor'] == 'цоколь', 'amnt_floor'] = 1
df_train.loc[df_train['floor'] == 'подвал', 'amnt_floor'] = 1
df_train.loc[df_train['floor'] == '1 этаж', 'amnt_floor'] = 1
df_train.loc[df_train['floor'] == '4 этаж', 'amnt_floor'] = 1
df_train.loc[df_train['floor'] == '1-й', 'amnt_floor'] = 1
df_train.loc[df_train['floor'] == '3 этаж', 'amnt_floor'] = 1
df_train.loc[df_train['floor'] == 'фактически на уровне 1 этажа', 'amnt_floor'] = 1
df_train.loc[df_train['floor'] == '1 (Цокольный этаж)', 'amnt_floor'] = 1
df_train.loc[df_train['floor'] == '5 этаж', 'amnt_floor'] = 1
df_train.loc[df_train['floor'] == 'Цоколь', 'amnt_floor'] = 1
df_train.loc[df_train['floor'] == 'Подвал', 'amnt_floor'] = 1
df_train.loc[df_train['floor'] == '1, антресоль', 'amnt_floor'] = 1.5
df_train.loc[df_train['floor'] == '5, мансарда', 'amnt_floor'] = 1
df_train.loc[df_train['floor'] == '3 (антресоль)', 'amnt_floor'] = 1
df_train.loc[df_train['floor'] == 'мансарда (4эт)', 'amnt_floor'] = 1
df_train.loc[df_train['floor'] == 'тех.этаж (6)', 'amnt_floor'] = 1
df_train.loc[df_train['floor'] == '1,2,3,4', 'amnt_floor'] = 4
df_train.loc[df_train['floor'] == 'цоколь, 1', 'amnt_floor'] = 2
df_train.loc[df_train['floor'] == 'подвал , 1', 'amnt_floor'] = 2
df_train.loc[df_train['floor'] == '1 этаж, подвал', 'amnt_floor'] = 2
df_train.loc[df_train['floor'] == 'подвал, 1', 'amnt_floor'] = 2
df_train.loc[df_train['floor'] == '1,2,3', 'amnt_floor'] = 3
df_train.loc[df_train['floor'] == '2,3', 'amnt_floor'] = 2
df_train.loc[df_train['floor'] == 'подва, 1.2 этаж', 'amnt_floor'] = 2
df_train.loc[df_train['floor'] == '1-й, 2-й', 'amnt_floor'] = 2
df_train.loc[df_train['floor'] == '1,2,3,4,5', 'amnt_floor'] = 5
df_train.loc[df_train['floor'] == '3, Мансарда (4 эт)', 'amnt_floor'] = 2
df_train.loc[df_train['floor'] == '1-3', 'amnt_floor'] = 3
df_train.loc[df_train['floor'] == 'подвал, 3. 4 этаж', 'amnt_floor'] = 3
df_train.loc[df_train['floor'] == '1 + подвал (без отделки)', 'amnt_floor'] = 2
df_train.loc[df_train['floor'] == 'подвал , 1 ', 'amnt_floor'] = 2
df_train.loc[df_train['floor'] == '4, 5', 'amnt_floor'] = 2
df_train.loc[df_train['floor'] == '1, 2.', 'amnt_floor'] = 2
df_train.loc[df_train['floor'] == '2, 3', 'amnt_floor'] = 2
df_train.loc[df_train['floor'] == '1,2,антресоль', 'amnt_floor'] = 2.5
df_train.loc[df_train['floor'] == 'подвал, 1-7, техэтаж', 'amnt_floor'] = 9
df_train.loc[df_train['floor'] == '7,8', 'amnt_floor'] = 2
df_train.loc[df_train['floor'] == 'подвал, 1-4 этаж', 'amnt_floor'] = 5
df_train.loc[df_train['floor'] == '3, мансарда', 'amnt_floor'] = 4
df_train.loc[df_train['floor'] == '1, подвал, антресоль', 'amnt_floor'] = 2.5
df_train.loc[df_train['floor'] == 'подвал, цоколь, 1 этаж', 'amnt_floor'] = 3
df_train.loc[df_train['floor'] == '1.2', 'amnt_floor'] = 2
df_train.loc[df_train['floor'] == 'подвал, 1-3', 'amnt_floor'] = 4
df_train.loc[df_train['floor'] == 'подвал,1', 'amnt_floor'] = 2
df_train.loc[df_train['floor'] == 'подвал, 1,2,3', 'amnt_floor'] = 4
df_train.loc[df_train['floor'] == '1-й, подвал', 'amnt_floor'] = 2
df_train.loc[df_train['floor'] == 'мансарда', 'amnt_floor'] = 1
df_train.loc[df_train['floor'] == 'мезонин', 'amnt_floor'] = 1

## Train Кодирование признака floor 

In [25]:
df_train.loc[df_train['floor'] == -1.1, 'floor'] = -1
df_train.loc[df_train['floor'] == '1, 2, 3', 'floor'] = 2
df_train.loc[df_train['floor'] == '1,2', 'floor'] = 1.5
df_train.loc[df_train['floor'] == '1, подвал', 'floor'] = 0.5
df_train.loc[df_train['floor'] == 'цоколь', 'floor'] = 0
df_train.loc[df_train['floor'] == 'подвал', 'floor'] = -1
df_train.loc[df_train['floor'] == '1 этаж', 'floor'] = 1
df_train.loc[df_train['floor'] == '4 этаж', 'floor'] = 4
df_train.loc[df_train['floor'] == '1-й', 'floor'] = 1
df_train.loc[df_train['floor'] == '3 этаж', 'floor'] = 3
df_train.loc[df_train['floor'] == 'фактически на уровне 1 этажа', 'floor'] = 1 
df_train.loc[df_train['floor'] == '1 (Цокольный этаж)', 'floor'] = 0
df_train.loc[df_train['floor'] == '5 этаж', 'floor'] = 5
df_train.loc[df_train['floor'] == 'Цоколь', 'floor'] = 0
df_train.loc[df_train['floor'] == 'Подвал', 'floor'] = -1
df_train.loc[df_train['floor'] == '1, антресоль', 'floor'] = 1 
df_train.loc[df_train['floor'] == '5, мансарда', 'floor'] = 5
df_train.loc[df_train['floor'] == '3 (антресоль)', 'floor'] = 3
df_train.loc[df_train['floor'] == 'мансарда (4эт)', 'floor'] = 4
df_train.loc[df_train['floor'] == 'тех.этаж (6)', 'floor'] = 6
df_train.loc[df_train['floor'] == '1,2,3,4', 'floor'] = 2.5
df_train.loc[df_train['floor'] == 'цоколь, 1', 'floor'] = 0.5
df_train.loc[df_train['floor'] == 'подвал , 1', 'floor'] = 0.5
df_train.loc[df_train['floor'] == '1 этаж, подвал', 'floor'] = 0.5
df_train.loc[df_train['floor'] == 'Цоколь, 1,2(мансарда)', 'floor'] = 1
df_train.loc[df_train['floor'] == 'подвал , 1', 'floor'] = 0.5
df_train.loc[df_train['floor'] == '1, цоколь', 'floor'] = 0.5
df_train.loc[df_train['floor'] == 'подвал,1,2,3,4,5', 'floor'] = 2.5
df_train.loc[df_train['floor'] == '1, 2', 'floor'] = 1.5
df_train.loc[df_train['floor'] == 'подвал , 1', 'floor'] = 0.5
df_train.loc[df_train['floor'] == '1 этаж, подвал', 'floor'] = 0.5
df_train.loc[df_train['floor'] == 'подвал, 1', 'floor'] = 0.5
df_train.loc[df_train['floor'] == '1,2,3', 'floor'] = 2
df_train.loc[df_train['floor'] == '2,3', 'floor'] = 2.5
df_train.loc[df_train['floor'] == 'подва, 1.2 этаж', 'floor'] = 1
df_train.loc[df_train['floor'] == '1-й, 2-й', 'floor'] = 1.5
df_train.loc[df_train['floor'] == '1,2,3,4,5', 'floor'] = 3
df_train.loc[df_train['floor'] == '3, Мансарда (4 эт)', 'floor'] = 3.5
df_train.loc[df_train['floor'] == '1-3', 'floor'] = 2
df_train.loc[df_train['floor'] == 'подвал, 3. 4 этаж', 'floor'] = 3
df_train.loc[df_train['floor'] == '1 + подвал (без отделки)', 'floor'] = 0.5
df_train.loc[df_train['floor'] == 'подвал , 1 ', 'floor'] = 0.5
df_train.loc[df_train['floor'] == '4, 5', 'floor'] = 4.5
df_train.loc[df_train['floor'] == '1, 2.', 'floor'] = 1.5
df_train.loc[df_train['floor'] == '2, 3', 'floor'] = 2.5
df_train.loc[df_train['floor'] == '1,2,антресоль', 'floor'] = 2
df_train.loc[df_train['floor'] == 'подвал, 1-7, техэтаж', 'floor'] = 4.5
df_train.loc[df_train['floor'] == '7,8', 'floor'] = 7.5
df_train.loc[df_train['floor'] == 'подвал, 1-4 этаж', 'floor'] = 3
df_train.loc[df_train['floor'] == '3, мансарда', 'floor'] = 3.5
df_train.loc[df_train['floor'] == '1, подвал, антресоль', 'floor'] = 0.5
df_train.loc[df_train['floor'] == 'подвал, цоколь, 1 этаж', 'floor'] = 0
df_train.loc[df_train['floor'] == '1.2', 'floor'] = 1.5
df_train.loc[df_train['floor'] == 'подвал, 1-3', 'floor'] = 1.5
df_train.loc[df_train['floor'] == 'подвал,1', 'floor'] = 0.5
df_train.loc[df_train['floor'] == 'подвал, 1,2,3', 'floor'] = 1.5
df_train.loc[df_train['floor'] == '1-й, подвал', 'floor'] = 0.5
df_train.loc[df_train['floor'] == 'мансарда', 'floor'] = -1.1
df_train.loc[df_train['floor'] == 'мезонин', 'floor'] = -1.1

### Test признак с количеством этажей

In [26]:
df_test['amnt_floor'] = None

In [27]:
df_test.floor.unique()

array(['1.0', '-1.0', nan, '2,3', '2', '-2.0', '5.0', '4', '1', '5',
       '1, 2', '1,2,3', '2.0', '4.0', '1,2,3,4', 'цоколь', '3', 'подвал',
       '3.0', '11.0', '9.0', '12.0', '0.0', 'цоколь, 1, 2,3,4,5,6', '6',
       '1,2', ' 1, 2, Антресоль', '6.0', '3 этаж, мансарда (4 этаж)',
       'цокольный', '1-й, 2-й', '13', '1, подвал', '7', '1, 2, 3', '1,2 ',
       '14', '9', 'подвал,1', '1-й', '8', '3,4', 'мансарда',
       'подвал, 1 и 4 этаж', '10.0', '5(мансарда)',
       'технический этаж,5,6', ' 1-2, подвальный', '1, 2, 3, мансардный',
       '8.0', '7.0', 'подвал, 1, 2, 3',
       '1,2,3, антресоль, технический этаж', '3, 4', '4, 5',
       '1-3 этажи, цоколь (188,4 кв.м), подвал (104 кв.м)',
       '1,2,3,4, подвал', '2-й', '1, 2 этаж', '1,2,3,4,5', 'подвал, 1, 2',
       '1-7', '1 (по док-м цоколь)', '1-й, подвал', '10', '1,2,подвал ',
       '13.0', 'подвал, 2', '1, цоколь', 'подвал,1,2,3',
       '1,2,3 этаж, подвал', 'цоколь, 1', '2, 3, 4, тех.этаж',
       'цокольный, 1,2'

In [28]:
df_test.loc[df_test['floor'] == '1', 'amnt_floor'] = 1
df_test.loc[df_test['floor'] == '-1.0', 'amnt_floor'] = 1
df_test.loc[df_test['floor'] == '2', 'amnt_floor'] = 1
df_test.loc[df_test['floor'] == '-2.0', 'amnt_floor'] = 1
df_test.loc[df_test['floor'] == '5.0', 'amnt_floor'] = 1
df_test.loc[df_test['floor'] == '4', 'amnt_floor'] = 1
df_test.loc[df_test['floor'] == '1', 'amnt_floor'] = 1
df_test.loc[df_test['floor'] == '5', 'amnt_floor'] = 1
df_test.loc[df_test['floor'] == '2.0', 'amnt_floor'] = 1
df_test.loc[df_test['floor'] == '4.0', 'amnt_floor'] = 1
df_test.loc[df_test['floor'] == '3', 'amnt_floor'] = 1
df_test.loc[df_test['floor'] == '3.0', 'amnt_floor'] = 1
df_test.loc[df_test['floor'] == '11.0', 'amnt_floor'] = 1
df_test.loc[df_test['floor'] == '9.0', 'amnt_floor'] = 1
df_test.loc[df_test['floor'] == '12.0', 'amnt_floor'] = 1
df_test.loc[df_test['floor'] == '0.0', 'amnt_floor'] = 1
df_test.loc[df_test['floor'] == '6', 'amnt_floor'] = 1
df_test.loc[df_test['floor'] == '6.0', 'amnt_floor'] = 1
df_test.loc[df_test['floor'] == '13', 'amnt_floor'] = 1
df_test.loc[df_test['floor'] == '7', 'amnt_floor'] = 1
df_test.loc[df_test['floor'] == '14', 'amnt_floor'] = 1
df_test.loc[df_test['floor'] == '9', 'amnt_floor'] = 1
df_test.loc[df_test['floor'] == '8', 'amnt_floor'] = 1
df_test.loc[df_test['floor'] == '10.0', 'amnt_floor'] = 1
df_test.loc[df_test['floor'] == '8.0', 'amnt_floor'] = 1
df_test.loc[df_test['floor'] == '7.0', 'amnt_floor'] = 1
df_test.loc[df_test['floor'] == '10', 'amnt_floor'] = 1
df_test.loc[df_test['floor'] == '13.0', 'amnt_floor'] = 1

In [30]:
df_test.loc[df_test['floor'] == '1,2 ', 'amnt_floor'] = 2
df_test.loc[df_test['floor'] == '1,2,подвал ', 'amnt_floor'] = 3
df_test.loc[df_test['floor'] == 'цоколь', 'amnt_floor'] = 1
df_test.loc[df_test['floor'] == 'подвал', 'amnt_floor'] = 1
df_test.loc[df_test['floor'] == '1,2', 'amnt_floor'] = 2
df_test.loc[df_test['floor'] == 'цокольный', 'amnt_floor'] = 1
df_test.loc[df_test['floor'] == 'Техническое подполье', 'amnt_floor'] = 1
df_test.loc[df_test['floor'] == '1-й', 'amnt_floor'] = 1
df_test.loc[df_test['floor'] == '1, 2, 3', 'amnt_floor'] = 3
df_test.loc[df_test['floor'] == '5(мансарда)', 'amnt_floor'] = 1
df_test.loc[df_test['floor'] == '1,2,3 этаж, подвал', 'amnt_floor'] = 4
df_test.loc[df_test['floor'] == '1, подвал', 'amnt_floor'] = 2
df_test.loc[df_test['floor'] == '1, 2', 'amnt_floor'] = 2
df_test.loc[df_test['floor'] == '1, 2, 3', 'amnt_floor'] = 3
df_test.loc[df_test['floor'] == '1,2,3 этаж, подвал', 'amnt_floor'] = 4
df_test.loc[df_test['floor'] == 'цоколь, 1, 2,3,4,5,6', 'amnt_floor'] = 7
df_test.loc[df_test['floor'] == '1,2,3', 'amnt_floor'] = 3
df_test.loc[df_test['floor'] == '3, 4', 'amnt_floor'] = 2
df_test.loc[df_test['floor'] == '1-й, подвал', 'amnt_floor'] = 2
df_test.loc[df_test['floor'] == 'подвал,1', 'amnt_floor'] = 2
df_test.loc[df_test['floor'] == '1 (по док-м цоколь)', 'amnt_floor'] = 1
df_test.loc[df_test['floor'] == '1,2,3, антресоль, технический этаж', 'amnt_floor'] = 4
df_test.loc[df_test['floor'] == '1-й, 2-й', 'amnt_floor'] = 2
df_test.loc[df_test['floor'] == 'подвал,1,2,3', 'amnt_floor'] = 4
df_test.loc[df_test['floor'] == '1,2', 'amnt_floor'] = 2
df_test.loc[df_test['floor'] == '1,2,3,4,5', 'amnt_floor'] = 5
df_test.loc[df_test['floor'] == '3 этаж, мансарда (4 этаж)', 'amnt_floor'] = 2
df_test.loc[df_test['floor'] == 'подвал, 1, 2', 'amnt_floor'] = 3
df_test.loc[df_test['floor'] == '1-7', 'amnt_floor'] = 7
df_test.loc[df_test['floor'] == '3,4', 'amnt_floor'] = 2
df_test.loc[df_test['floor'] == '1,2,подвал', 'amnt_floor'] = 3
df_test.loc[df_test['floor'] == '2-й', 'amnt_floor'] = 1
df_test.loc[df_test['floor'] == '1-3 этажи, цоколь (188,4 кв.м), подвал (104 кв.м)', 'amnt_floor'] = 5
df_test.loc[df_test['floor'] == '1,2,подвал', 'amnt_floor'] = 3
df_test.loc[df_test['floor'] == '1, 2 этаж', 'amnt_floor'] = 2
df_test.loc[df_test['floor'] == 'мансарда', 'amnt_floor'] = 1
df_test.loc[df_test['floor'] == '1, цоколь', 'amnt_floor'] = 2
df_test.loc[df_test['floor'] == '1, 2, 3, мансардный', 'amnt_floor'] = 4
df_test.loc[df_test['floor'] == 'подвал, 1, 2, 3', 'amnt_floor'] = 4
df_test.loc[df_test['floor'] == '1,2,3,4', 'amnt_floor'] = 4
df_test.loc[df_test['floor'] == 'подвал, 1 и 4 этаж', 'amnt_floor'] = 5
df_test.loc[df_test['floor'] == '1,2,3,4, подвал', 'amnt_floor'] = 5
df_test.loc[df_test['floor'] == 'цокольный, 1,2, подвал', 'amnt_floor'] = 4
df_test.loc[df_test['floor'] == 'технический этаж,5,6', 'amnt_floor'] = 3
df_test.loc[df_test['floor'] == '2,3', 'amnt_floor'] = 2
df_test.loc[df_test['floor'] == ' 1-2, подвальный', 'amnt_floor'] = 3
df_test.loc[df_test['floor'] == 'подвал, 2', 'amnt_floor'] = 2
df_test.loc[df_test['floor'] == '1,2', 'amnt_floor'] = 2
df_test.loc[df_test['floor'] == 'цоколь, 1', 'amnt_floor'] = 2
df_test.loc[df_test['floor'] == '2, 3, 4, тех.этаж', 'amnt_floor'] = 4
df_test.loc[df_test['floor'] == '4, 5', 'amnt_floor'] = 2
df_test.loc[df_test['floor'] == ' 1, 2, Антресоль', 'amnt_floor'] = 2.5
df_test.loc[df_test['floor'] == '1,2', 'amnt_floor'] = 2
df_test.loc[df_test['floor'] == '1,2,подвал', 'amnt_floor'] = 3
df_test.loc[df_test['floor'] == 'цокольный, 1,2', 'amnt_floor'] = 3

### Test кодирование признака floor


In [32]:
df_test.loc[df_test['floor'] == '1,2 ', 'floor'] = 1.5
df_test.loc[df_test['floor'] == '1,2,подвал ', 'floor'] = 1
df_test.loc[df_test['floor'] == 'цоколь', 'floor'] = 0
df_test.loc[df_test['floor'] == 'подвал', 'floor'] = -1
df_test.loc[df_test['floor'] == '1,2', 'floor'] = 1.5
df_test.loc[df_test['floor'] == 'цокольный', 'floor'] = 0
df_test.loc[df_test['floor'] == 'Техническое подполье', 'floor'] = -1
df_test.loc[df_test['floor'] == '1-й', 'floor'] = 1
df_test.loc[df_test['floor'] == '1, 2, 3', 'floor'] = 2
df_test.loc[df_test['floor'] == '5(мансарда)', 'floor'] = 5
df_test.loc[df_test['floor'] == '1,2,3 этаж, подвал', 'floor'] = 1.5
df_test.loc[df_test['floor'] == '1, подвал', 'floor'] = 0.5
df_test.loc[df_test['floor'] == '1, 2', 'floor'] = 1.5
df_test.loc[df_test['floor'] == '1, 2, 3', 'floor'] = 2
df_test.loc[df_test['floor'] == '1,2,3 этаж, подвал', 'floor'] = 1.5
df_test.loc[df_test['floor'] == 'цоколь, 1, 2,3,4,5,6', 'floor'] = 3
df_test.loc[df_test['floor'] == '1,2,3', 'floor'] = 2
df_test.loc[df_test['floor'] == '3, 4', 'floor'] = 3.5
df_test.loc[df_test['floor'] == '1-й, подвал', 'floor'] = 0.5
df_test.loc[df_test['floor'] == 'подвал,1', 'floor'] = 0.5
df_test.loc[df_test['floor'] == '1 (по док-м цоколь)', 'floor'] = 0
df_test.loc[df_test['floor'] == '1,2,3, антресоль, технический этаж', 'floor'] = 2.5
df_test.loc[df_test['floor'] == '1-й, 2-й', 'floor'] = 1.5
df_test.loc[df_test['floor'] == 'подвал,1,2,3', 'floor'] = 1.5
df_test.loc[df_test['floor'] == '1,2', 'floor'] = 1.5
df_test.loc[df_test['floor'] == '1,2,3,4,5', 'floor'] = 3
df_test.loc[df_test['floor'] == '3 этаж, мансарда (4 этаж)', 'floor'] = 3.5
df_test.loc[df_test['floor'] == 'подвал, 1, 2', 'floor'] = 1
df_test.loc[df_test['floor'] == '1-7', 'floor'] = 4
df_test.loc[df_test['floor'] == '3,4', 'floor'] = 3.5
df_test.loc[df_test['floor'] == '1,2,подвал', 'floor'] = 1
df_test.loc[df_test['floor'] == '2-й', 'floor'] = 2
df_test.loc[df_test['floor'] == '1-3 этажи, цоколь (188,4 кв.м), подвал (104 кв.м)', 'floor'] = 1
df_test.loc[df_test['floor'] == '1,2,подвал', 'floor'] = 1
df_test.loc[df_test['floor'] == '1, 2 этаж', 'floor'] = 1.5
df_test.loc[df_test['floor'] == 'мансарда', 'floor'] = -1.1
df_test.loc[df_test['floor'] == '1, цоколь', 'floor'] = 0.5
df_test.loc[df_test['floor'] == '1, 2, 3, мансардный', 'floor'] = 2.5
df_test.loc[df_test['floor'] == 'подвал, 1, 2, 3', 'floor'] = 1.5
df_test.loc[df_test['floor'] == '1,2,3,4', 'floor'] = 2.5
df_test.loc[df_test['floor'] == 'подвал, 1 и 4 этаж', 'floor'] = 2
df_test.loc[df_test['floor'] == '1,2,3,4, подвал', 'floor'] = 2
df_test.loc[df_test['floor'] == 'цокольный, 1,2, подвал', 'floor'] = 1
df_test.loc[df_test['floor'] == 'технический этаж,5,6', 'floor'] = 5
df_test.loc[df_test['floor'] == '2,3', 'floor'] = 2.5
df_test.loc[df_test['floor'] == ' 1-2, подвальный', 'floor'] = 1
df_test.loc[df_test['floor'] == 'подвал, 2', 'floor'] = 1
df_test.loc[df_test['floor'] == '1,2', 'floor'] = 1.5
df_test.loc[df_test['floor'] == 'цоколь, 1', 'floor'] = 0.5
df_test.loc[df_test['floor'] == '2, 3, 4, тех.этаж', 'floor'] = 3.5
df_test.loc[df_test['floor'] == '4, 5', 'floor'] = 4.5
df_test.loc[df_test['floor'] == ' 1, 2, Антресоль', 'floor'] = 2
df_test.loc[df_test['floor'] == '1,2', 'floor'] = 1.5
df_test.loc[df_test['floor'] == '1,2,подвал', 'floor'] = 1
df_test.loc[df_test['floor'] == 'цокольный, 1,2', 'floor'] = 1

Переведем значения признаков **floor** и **amnt_floor** в float

In [37]:
df_train['floor'] = df_train['floor'].astype(float)
df_train['amnt_floor'] = df_train['amnt_floor'].astype(float)

df_test['floor'] = df_test['floor'].astype(float)
df_test['amnt_floor'] = df_test['amnt_floor'].astype(float)

Создадим признак **month**, чтобы группировать данные по месяцам при кросс-валидации, а **date** удалим

In [38]:
df_train['month'] = pd.DatetimeIndex(df_train['date']).month
df_test['month'] = pd.DatetimeIndex(df_test['date']).month

df_train.drop('date', axis=1, inplace=True)
df_test.drop('date', axis=1, inplace=True)

## Кодирование категориальных признаков

Для кодирования категориальных признаков будем использовать алгоритм **Target Encoding**

In [40]:
import typing
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.exceptions import NotFittedError

class SmoothedTargetEncoding(BaseEstimator,TransformerMixin):
    """Регуляризованный таргет энкодинг.

    :param categorical_features: список из столбцов с категориальными признаками, которые нужно заэнкодить
    :param alpha: параметр регуляризации
    """

    def __init__(self, categorical_features: typing.List[str], alpha: float = 50.0):
        self.__is_fitted = False
        self.categorical_features = categorical_features
        self.alpha = alpha
        self.mean_price = None
        self.mean_price_by_cat = {}
        self.encoded_preffix = "encoded_"
        self.target = 'target'

    def smoothed_target_encoding(self, y: pd.Series) -> pd.Series:
        """Реализация регуляризованного таргед энкодинга.

        Принцип такой - чем меньше исходных данных, тем сильнее будет регуляризация
        Параметр регуляризации регуляризует мин. кол-во необходимых данных
        :param y: pd.Series с ценой
        :return: pd.Series с регуляризованной ценой
        """
        nrows = y.notnull().sum()
        return (y.mean() * nrows + self.alpha * self.mean_price) / (nrows + self.alpha)

    def fit(self, X: pd.DataFrame, y: typing.Union[np.array, pd.Series] = None):
        """На основе обучающей выборки запоминает средние цены в разрезе категорий.

        :param X: pd.DataFrame, выборка из категориальных признаков
        :param y: target
        :return:
        """
        X[self.target] = y
        self.mean_price = X[self.target].mean()
        for col in self.categorical_features:
            self.mean_price_by_cat[col] = (
                X.groupby(col)[self.target].apply(lambda x: self.smoothed_target_encoding(x)).fillna(self.mean_price)
            )

        X.drop(self.target, axis=1, inplace=True)
        self.__is_fitted = True
        return self

    def transform(self, X: pd.DataFrame, y: typing.Union[np.array, pd.Series] = None):
        """Применение регуляризованного таргет энкодинга.

        :param X: pd.DataFrame, выборка из категориальных признаков
        :return: pd.DataFrame, закодированная выборка
        """
        X_cp = X.copy()
        if self.__is_fitted:
            encoded_cols = []
            for col in self.categorical_features:
                new_col = self.encoded_preffix + col
                X_cp[new_col] = X_cp[col].map(self.mean_price_by_cat[col]).fillna(self.mean_price)
                encoded_cols.append(new_col)
            return X_cp[encoded_cols]
        else:
            raise NotFittedError(
                "This {} instance is not fitted yet. Call 'fit' with appropriate arguments before using this transformer".format(
                    type(self).__name__
                )
            )

In [41]:
# категориальные признаки
categorical = df_train.columns[df_train.dtypes == 'object']

In [42]:
# иинциализация энкодера
encoder = SmoothedTargetEncoding(df_train[categorical].columns.to_list())

# обучение энкодера
encoder.fit(pd.concat([df_train, df_test]), df_train['per_square_meter_price'])

# кодирование категориальных признаков
encoded = encoder.transform(pd.concat([df_train, df_test]), df_train['per_square_meter_price'])

In [43]:
encoded

,encoded_city,encoded_osm_city_nearest_name,encoded_region,encoded_street
id,,,,
COL_62,72678.799595,71870.082281,71866.025545,72081.910401
COL_71,54302.836712,54302.836712,54346.996221,65918.963210
COL_140,72678.799595,71870.082281,71866.025545,69069.930097
COL_202,70098.040766,69205.210090,64704.085510,67823.093751
COL_207,54325.773969,54195.875112,53662.593055,66589.044235
...,...,...,...,...
COL_455089,72678.799595,71870.082281,71866.025545,67747.100260
COL_455212,52989.054741,52681.526654,52864.946496,69012.011925
COL_455261,55448.669383,55386.177957,54487.717833,69158.611507


In [44]:
# числовые признаки
numerical = df_test.columns[df_test.dtypes != 'object']

# выборка из чиловых признаков
numerical_df = pd.concat([df_train[numerical], df_test[numerical]])

# объединим закодированную категориальную и числовую выборки
final_df = pd.concat([numerical_df, encoded], axis=1)

## Генерация новых признаков

Сгенерируем новые признаки путем агрегирования имеющихся, на основе признаков с самой высокой корреляцией с целевой переменной, помимо этого пробовали это делать на основе самых важных признаков, после обучения первой модели, но этот подход показал себя хуже.

In [45]:
# добавим в новый датафрейм целевую переменную
final_df['target'] = df_train['per_square_meter_price']

In [46]:
# признаки с самой высокой корреляцией с целевой переменной
final_df.corr()['target'].sort_values(ascending=False).head(20)

target                           1.000000
encoded_street                   0.548010
osm_catering_points_in_0.0075    0.530931
osm_hotels_points_in_0.005       0.530383
osm_hotels_points_in_0.0075      0.529142
osm_hotels_points_in_0.01        0.528754
osm_catering_points_in_0.01      0.528271
osm_catering_points_in_0.005     0.524550
encoded_region                   0.515099
osm_offices_points_in_0.005      0.512835
encoded_city                     0.511094
osm_offices_points_in_0.01       0.507586
encoded_osm_city_nearest_name    0.503054
osm_amenity_points_in_0.01       0.502173
osm_offices_points_in_0.0075     0.501714
osm_amenity_points_in_0.0075     0.495242
osm_catering_points_in_0.001     0.488075
osm_amenity_points_in_0.005      0.487470
osm_shops_points_in_0.01         0.486203
osm_healthcare_points_in_0.01    0.474950
Name: target, dtype: float64

In [48]:
def agg_features(main_columns, group_columns, aggregations, df):
  """Создание новых агрегированных признаков

  :param main_columns: list, числовые признаки, по которым будет агрегация
  :param group_columns: list, категориальные признаки, по которым будет группировка данных
  :param aggregations: list, функция агрегирования
  :param df: pd.DataFrame, датафрейм, в котором создаются новые признаки
  :return: pd.DataFrame, датафрейм с новыми агрегированными признаквами
  """
  for main_column in main_columns:  
      for col in group_columns:
          for agg in aggregations:
              new_col_name = f'{main_column}_{col}_{agg}'
                
              new_df = df.groupby([col])[main_column].agg([agg]).reset_index().rename(columns={agg: new_col_name})

              new_df.index = list(new_df[col])
              new_df = new_df[new_col_name].to_dict()   

              df[new_col_name] = df[col].map(new_df).astype('float32')

              print(f'{new_col_name}, ', end='')  

In [49]:
agg_features(['osm_catering_points_in_0.0075', 
           'osm_hotels_points_in_0.005',
           'osm_offices_points_in_0.005',
           'osm_shops_points_in_0.01'],
          ['encoded_street', 'encoded_region', 'encoded_city'], ['mean'], final_df)

osm_catering_points_in_0.0075_encoded_street_mean, osm_catering_points_in_0.0075_encoded_region_mean, osm_catering_points_in_0.0075_encoded_city_mean, osm_hotels_points_in_0.005_encoded_street_mean, osm_hotels_points_in_0.005_encoded_region_mean, osm_hotels_points_in_0.005_encoded_city_mean, osm_offices_points_in_0.005_encoded_street_mean, osm_offices_points_in_0.005_encoded_region_mean, osm_offices_points_in_0.005_encoded_city_mean, osm_shops_points_in_0.01_encoded_street_mean, osm_shops_points_in_0.01_encoded_region_mean, osm_shops_points_in_0.01_encoded_city_mean, 

In [50]:
# удаление целевой переменной и price_type
final_df.drop('target', axis=1, inplace=True)
final_df.drop('price_type', axis=1, inplace=True)

## Заполнение пропусков

Заполним пропуски в числовых признаках константой -1.1 и укажем это в гиперпараметрах xgboost, попытки заполнить пропуски медианой или средним, группируя по региону показали результаты хуже

In [51]:
def num_fill_na(df, grouping_col, nan_cols, agg):
  """
  заполнение пропусков в числовых признаков, группировкой по категориальному признаку
  :param df: pd.DataFrame, датафрейм с пропусками
  :param grouping_col: string, признак, по которому группируется датафрейм
  :param nan_cols: list, список из признаков с пропусками
  :param agg: функция агрегации
  :return: pd.DataFrame, датафрейм с заполненными пропусками
  """
  for col in nan_cols:
    df[col] = df[col].fillna(df.groupby(grouping_col)[col].transform(agg))

  return df

In [59]:
final_df = final_df.fillna(-1.1)

print(f'В итоговом датафрейме {final_df.isna().sum().sum()} пропусков.')

В итоговом датафрейме 0 пропусков.


## Кластеризация по долготе и широте

В датасете признаки **lat** и **lng** отвечают за долготу и широту наблюдаемого объекта с небольшим шумом, сами по себе пользы для модели они не несут, но создание на их основе нового признака с разбиеним их на 8 кластеров(именно столько федеральных округов в России) улучшило результат. Для этого мы используем алгоритм **KMeans**

In [60]:
from sklearn.cluster import KMeans

In [61]:
# инициализация алгоритма
k = KMeans(n_clusters=8, random_state=23)

# обучение
k.fit(final_df[['lat', 'lng']])

# прогноз
clusters = k.predict(final_df[['lat', 'lng']])

In [62]:
# новый признак с кластерами
final_df['lat_lng_clusters'] = clusters

# удаление lat и lng
final_df.drop(['lat', 'lng'], axis=1, inplace=True)

## Метрика 

Кастомная метрика хакатона

In [63]:
import typing
import numpy as np
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_percentage_error

THRESHOLD = 0.15
NEGATIVE_WEIGHT = 1.1

def deviation_metric_one_sample(y_true: typing.Union[float, int], y_pred: typing.Union[float, int]) -> float:
    """
    Реализация кастомной метрики для хакатона.

    :param y_true: float, реальная цена
    :param y_pred: float, предсказанная цена
    :return: float, значение метрики
    """
    deviation = (y_pred - y_true) / np.maximum(1e-8, y_true)
    if np.abs(deviation) <= THRESHOLD:
        return 0
    elif deviation <= - 4 * THRESHOLD:
        return 9 * NEGATIVE_WEIGHT
    elif deviation < -THRESHOLD:
        return NEGATIVE_WEIGHT * ((deviation / THRESHOLD) + 1) ** 2
    elif deviation < 4 * THRESHOLD:
        return ((deviation / THRESHOLD) - 1) ** 2
    else:
        return 9

def deviation_metric(y_true: np.array, y_pred: np.array) -> float:
    return np.array([deviation_metric_one_sample(y_true[n], y_pred[n]) for n in range(len(y_true))]).mean()

def median_absolute_percentage_error(y_true: np.array, y_pred: np.array) -> float:
    return np.median(np.abs(y_pred-y_true)/y_true)

def metrics_stat(y_true: np.array, y_pred: np.array) -> typing.Dict[str,float]:
    mape = mean_absolute_percentage_error(y_true, y_pred)
    mdape = median_absolute_percentage_error(y_true, y_pred)
    rmse = mean_squared_error(y_true, y_pred, squared=False)
    r2 = r2_score(y_true, y_pred)
    raif_metric = deviation_metric(y_true, y_pred)
    return {'mape':mape, 'mdape':mdape, 'rmse': rmse, 'r2': r2, 'raif_metric':raif_metric}

In [64]:
# разобьем выборку на трейн и на тест
X_train = final_df.iloc[:4493, :]
X_test = final_df.iloc[4493:, :]

# целевая переменная
y = df_train['per_square_meter_price']

## Обучение и прогноз

Мы обучим 8 моделей **XGBRegressor**, а затем усредним их прогноз. Прогнозировать тестовую выборку будем с **GroupKFold**, с месяцами в виде фолдов. На каждом шаге итерации будем обучаться на всех месяцах, кроме одного, который будет валидационным.Также на каждом шаге будем прогнозировать всю тестовую выборку, а результаты затем усредним.

In [68]:
from sklearn.model_selection import GroupKFold

In [69]:
groups = X_train['month'].unique()
group_kfold = GroupKFold(n_splits=8)

In [70]:
groups

array([1, 2, 3, 4, 5, 6, 7, 8])

In [71]:
group_kfold.get_n_splits(X_train, y, groups)

8

In [74]:
# датафрейм, в котором будем хранить прогнозы с каждой итерации
pred_test = pd.DataFrame()
p = 1

for train_index, val_index in group_kfold.split(X_train, y, X_train['month']):
  print(f'Валидационный месяц: {df_train.iloc[val_index].month.iloc[0]}')
  print(f'Train: {len(train_index)} наблюдений, Val: {len(val_index)} наблюдений')
  # разбиваем выборку на трейн и валидацию, логарифмируя целевую переменную
  X_trainn, X_val = X_train.iloc[train_index], X_train.iloc[val_index]
  y_train, y_val = np.log1p(y.iloc[train_index]), np.log1p(y.iloc[val_index])
  # инициализация модели
  xx = XGBRegressor(n_estimators=1000, max_depth=10, missing=-1.1, random_state=23, verbosity=0)
  # обучение
  xx.fit(X_trainn, y_train)
  # прогноз валидационной выборки
  pred_val = xx.predict(X_val)
  # метрики
  metrics = metrics_stat(np.expm1(y_val), np.expm1(pred_val))
  print(metrics, '\n')

  # прогноз тестовой выборки
  pred = xx.predict(X_test)
  # фиксируем проноз в датафрейм
  pred_test[f'pred_{p}'] = pred
  p+=1

Валидационный месяц: 2
Train: 3843 наблюдений, Val: 650 наблюдений
{'mape': 0.20744541394793617, 'mdape': 0.15084828804242797, 'rmse': 41928.50222404582, 'r2': 0.754880244310396, 'raif_metric': 1.0275274023146568} 

Валидационный месяц: 3
Train: 3866 наблюдений, Val: 627 наблюдений
{'mape': 0.20261053582512809, 'mdape': 0.1380272996615116, 'rmse': 38146.197197973845, 'r2': 0.786180354733735, 'raif_metric': 0.8970943978856303} 

Валидационный месяц: 1
Train: 3914 наблюдений, Val: 579 наблюдений
{'mape': 0.21744665845936123, 'mdape': 0.15989743712782611, 'rmse': 61442.12212598472, 'r2': 0.5137467020249653, 'raif_metric': 1.1999730505128907} 

Валидационный месяц: 5
Train: 3921 наблюдений, Val: 572 наблюдений
{'mape': 0.22860989589715452, 'mdape': 0.14252671925145383, 'rmse': 28295.55863326429, 'r2': 0.8202403054431953, 'raif_metric': 1.2662734425079516} 

Валидационный месяц: 6
Train: 3932 наблюдений, Val: 561 наблюдений
{'mape': 0.2038312322464242, 'mdape': 0.1534848359089517, 'rmse': 8

In [97]:
# прогнозы моделей
np.expm1(pred_test)

,pred_1,pred_2,pred_3,pred_4,pred_5,pred_6,pred_7,pred_8
0,50060.703125,50073.117188,49941.820312,54619.160156,53036.960938,34587.906250,52801.777344,50124.718750
1,44840.007812,44006.562500,40849.285156,43652.976562,43328.832031,43252.203125,43347.554688,43119.175781
2,47607.664062,44030.406250,44300.269531,58247.527344,56313.929688,34783.273438,51339.894531,45215.886719
3,110202.250000,106206.875000,115943.484375,103823.273438,107315.125000,112144.992188,103250.671875,115819.812500
4,51514.449219,50928.070312,50370.597656,50533.566406,51137.101562,51751.546875,50877.437500,50072.828125
...,...,...,...,...,...,...,...,...
2969,28118.158203,26670.222656,23325.101562,25742.445312,28023.496094,29910.144531,26214.511719,24197.539062
2970,52922.265625,54575.367188,54180.355469,54091.964844,54339.890625,49512.109375,54338.132812,54385.203125
2971,50680.964844,43334.164062,40887.523438,55278.500000,58178.187500,53807.695312,48929.664062,55911.023438
2972,40741.554688,40388.039062,41166.027344,43724.929688,37386.718750,43266.687500,41005.765625,42895.882812


In [98]:
# датафрейм с усредненным прогнозом
sub_df = pd.DataFrame({'id': X_test.index,
                       'per_square_meter_price': np.expm1(pred_test.mean(axis=1))})

На завершение усредним наш прогноз с прогнозом полученным на основе реализации бейзлайна организаторов:

In [99]:
pred_ant = pd.read_csv('/content/targets_03.csv')

In [100]:
sub_df['per_square_meter_price'] = (sub_df['per_square_meter_price'] + pred_ant['per_square_meter_price']) / 2

In [102]:
# сохраним прогноз
sub_df.to_csv('raif_final_sub.csv')

In [104]:
pip freeze > requirements.txt